# CLLM generation & curation tutorial

# Setup keys and experiment

In [ ]:
import sys
sys.path.append('src/')

from cllm.utils import *
from cllm.llm_gen import *
from cllm.data_loader import *

import pandas as pd
from copy import deepcopy
import time
from sklearn.model_selection import train_test_split


#############################################################
# API KEY SETUP INSTRUCTIONS
#############################################################

# for vllm
# api_key = "EMPTY"
# api_base = "http://localhost:8000/v1"

# for together
# api_key = "add together api key"
# api_base = "https://api.together.xyz/v1"


# for azure openai
# api_key = "EMPTY"
# api_base = "add azure deployment link"

# for openai
# api_key = "EMPTY"
# api_base = DO NOT INCLUDE

#############################################################

api_details = {
     "api_base": "https://api.together.xyz/v1",
     "api_version": "2023-07-01-preview",
     "api_key": "91a65a9f1f3d858e4d7f4de4fdf14fde879712b254c81ca07c0cacc707b59d82",
}


model_short_name = 'mixtral' # 'gpt-4' (do not use other short names)
model = "mistralai/Mixtral-8x7B-Instruct-v0.1" # "gpt4_20230815" (use name of your model deployment)
llm_serving='together' # supported 'azure_openai', 'together', 'vllm'

seed = 0
ns = 20 # n_samples per class. e.g. if binary = 40 samples (i.e. 20 per class)
dataset = 'compas'
n_synthetic=100 # just to test --- normall should be 1000
n_processes = 5

# STEP 1: Generation

## Get dataset

In [4]:
rcc = pd.read_csv("rcctrain.csv")
rcc.rename(columns={'Groups': 'y'}, inplace=True)
df_feat = rcc.drop('y', axis=1)
df_label = rcc["y"]
df = rcc
count_0 = (df['y'] == 0).sum()
count_1 = (df['y'] == 1).sum()

print(f"Count of y=0: {count_0}")
print(f"Count of y=1: {count_1}")
needed_synthetic = count_0 - count_1

Count of y=0: 140
Count of y=1: 64


In [5]:
#df_feat, df_label, df = get_data(dataset=dataset, seed=seed)
df_feat.to_csv("dffeat")
df_label.to_csv("df_label")
df.to_csv("df")

X_train, X_remain, y_train, y_remain = sample_and_split(df_feat, df_label, ns=ns, seed=seed)

X_val, X_test, y_val, y_test = train_test_split(
    X_remain, y_remain, test_size=0.5, random_state=seed
)


X_train_orig = deepcopy(X_train)
y_train_orig = deepcopy(y_train)

## Setup Prompt

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser


response_schemas = []

example_df = pd.concat([X_train_orig, y_train_orig], axis=1)

# Shuffle
example_df = example_df.sample(frac=1).reset_index(drop=True)


for idx, col in enumerate(list(example_df.columns)):
    if col == 'y':
        resp = ResponseSchema(name='y',
                        description=f"binary label, {col}", )
    else:
        resp = ResponseSchema(name=col,
                        description=f"feature column", )
    response_schemas.append(resp)

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()



generator_template = """\
You are a synthetic data generator. 
Your goal is to produce data which mirrors \
the given examples in causal structure and feature and label distributions \
but also produces as diverse samples as possible

I will give you real examples first

Leverage your knowledge about criminal recividsm to generate 1000 realistic but diverse samples. 

example data: {data}

{format_instructions}

DO NOT COPY THE EXAMPLES but generate realistic but new and diverse samples which have the correct label conditioned on the features.
"""


prompt = ChatPromptTemplate.from_template(template=generator_template)

## Generate using LLM

In [9]:
retries = 4  # Max retries you want to attempt
df_llm = pd.DataFrame(columns=example_df.columns)
iters = 0
while True:
    print(f"Iter {iters}")
    print(f"Currently {df_llm.shape[0]} samples")
    while retries > 0:
        try:

            if len(example_df)>20:
                ic_samples=20
            else:
                ic_samples=len(example_df)
            
            #print(f'Running {dataset}, {seed}, {model} --- {n_processes}')
            tmp_llm = llm_gen(prompt, generator_template, format_instructions, example_df, 
                            n_samples=needed_synthetic*2,
                            temperature=0.9,
                            max_tokens=1000, model=model, 
                            n_processes=n_processes,
                            ic_samples=ic_samples, 
                            llm_serving=llm_serving, 
                            api_details=api_details)
        
            
            break  # if successful, break out of the loop
        except Exception as e:
            time.sleep(120)
            print(f"Error: {e}. Retrying with reduced n_processes...")
            n_processes = int(n_processes/2)
            retries -= 1
            if n_processes < 1:
                print("Error: Minimum n_processes reached. Exiting...")
                break
    # try:
    tmp_df = tmp_llm.astype(example_df.dtypes)
    tmp_df = tmp_df[tmp_df['y'] == 1]
    df_llm = pd.concat([df_llm, tmp_df], ignore_index=True)
    if df_llm.shape[0] >= needed_synthetic:
        break
# except:
#     pass

Iter 0
Currently 0 samples
idx: 0
df_tmp   hippurate-mannitol Dibutylamine hippuric acid 2-mercaptobenzothiazole  \
0        1234567.891  23456.78901    3456.78912              456.789123   

  N-acetyl-glucosaminic acid 2-phenylacetamide lys-Ile/lys-leu  y  
0                56789.12345       678901.2345     78901.23456  0  
idx: 1
idx: 2
idx: 3
idx: 4
Current =  8 (8, 8)


c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:255: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tmp = df_tmp.append(df_check, ignore_index=True)
c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:255: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tmp = df_tmp.append(df_check, ignore_index=True)
c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:255: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tmp = df_tmp.append(df_check, ignore_index=True)
c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:255: FutureWarning: The fram

idx: 0
df_tmp   hippurate-mannitol Dibutylamine hippuric acid 2-mercaptobenzothiazole  \
0        2468912.567   123891.256     18764.123                1089.234   

  N-acetyl-glucosaminic acid 2-phenylacetamide lys-Ile/lys-leu  y  
0                 156789.546        456789.123      123456.789  0  
idx: 1
idx: 2
idx: 3
idx: 4
Current =  22 (22, 8)


c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:255: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tmp = df_tmp.append(df_check, ignore_index=True)
c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:255: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tmp = df_tmp.append(df_check, ignore_index=True)
c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:255: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tmp = df_tmp.append(df_check, ignore_index=True)
c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:255: FutureWarning: The fram

idx: 0
df_tmp Empty DataFrame
Columns: []
Index: []
idx: 1
idx: 2
idx: 3
idx: 4
Current =  37 (37, 8)


...

c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:255: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tmp = df_tmp.append(df_check, ignore_index=True)
c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:255: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tmp = df_tmp.append(df_check, ignore_index=True)
c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:255: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tmp = df_tmp.append(df_check, ignore_index=True)
c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:255: FutureWarning: The fram

idx: 0
df_tmp    hippurate-mannitol  Dibutylamine  hippuric acid  2-mercaptobenzothiazole  \
0         1456345.678     85843.321      28654.875                 1546.290   
1         1378230.189     78934.182      26789.385                 1432.001   
2         1512458.985     80932.554      23545.867                 1345.198   
3         1425366.801     75678.981      25678.382                 1456.890   
4         1398230.985     76543.210      24567.890                 1345.678   

   N-acetyl-glucosaminic acid  2-phenylacetamide  lys-Ile/lys-leu  y  
0                   52385.302         984521.982         4921.880  0  
1                   49875.324         945671.831         4532.101  0  
2                   56789.345         976543.210         5123.890  0  
3                   53456.567         956789.210         4789.982  0  
4                   51234.876         967893.125         4678.982  0  
idx: 1
idx: 2
idx: 3
idx: 4
Current =  145 (145, 8)


c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:255: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tmp = df_tmp.append(df_check, ignore_index=True)
c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:255: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tmp = df_tmp.append(df_check, ignore_index=True)
c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:255: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tmp = df_tmp.append(df_check, ignore_index=True)
c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:255: FutureWarning: The fram

idx: 0
df_tmp   hippurate-mannitol  Dibutylamine hippuric acid 2-mercaptobenzothiazole  \
0       234567.89012  345678.90123    1234.56789             17654.32109   
1       765432.10987  876543.21098   234.5678901             6789.012345   
2       123456.78901  234567.89012   34.56789012             5678.901234   
3       567890.12345   67890.12345    789.012345              890.123456   
4        78901.23456    8901.23456    901.234567              1234.56789   

  N-acetyl-glucosaminic acid 2-phenylacetamide lys-Ile/lys-leu  y  
0               356789.01234     1567890.12345   2345678.90123  1  
1                76543.21098      876543.21098    987654.32109  1  
2                67890.12345       78901.23456     89012.34567  1  
3                 901.234567        1234.56789      234.567890  0  
4                 234.567890        34.5678901      4.56789012  0  
idx: 1
idx: 2
idx: 3
Current =  162 (162, 8)
Done...
162 (162, 8)


c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:255: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tmp = df_tmp.append(df_check, ignore_index=True)
c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:255: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tmp = df_tmp.append(df_check, ignore_index=True)
c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:255: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tmp = df_tmp.append(df_check, ignore_index=True)
c:\Users\chitt\OneDrive\Documents\Desktop\2200\docker\docker\workspace\ta\CLLM\src\cllm\llm_gen.py:278: FutureWarning: The fram

In [10]:
df_llm


,hippurate-mannitol,Dibutylamine,hippuric acid,2-mercaptobenzothiazole,N-acetyl-glucosaminic acid,2-phenylacetamide,lys-Ile/lys-leu,y
0,5.432110e+04,6789.012345,543.210987,234.123456,56789.01234,7.890123e+05,8.901235e+04,1
1,5.432110e+04,6789.123456,123.456789,56789.012340,123456.78910,2.345679e+06,3.456789e+06,1
2,1.456201e+06,316429.508400,9854.210330,3542.184210,278641.73120,2.345202e+06,1.806456e+06,1
3,7.836414e+05,55516.371460,7839.932526,2250.008480,79531.75333,6.029282e+05,7.173814e+05,1
4,1.195384e+06,91160.618530,15365.425440,900.389529,136703.07270,2.000293e+06,1.068308e+06,1
...,...,...,...,...,...,...,...,...
86,1.389283e+05,238430.477600,1095.420133,16590.667360,293454.51210,1.475067e+06,1.893961e+06,1
87,7.460122e+05,301611.136100,2511.791341,11327.215380,381246.07240,1.203403e+06,1.628105e+06,1
88,1.551939e+05,127500.783200,1747.820155,7591.357666,225639.19150,7.886709e+05,1.020119e+06,1
89,5.468361e+05,452034.358200,5801.856165,26329.945880,819308.59020,2.530293e+06,3.529588e+06,1


In [11]:
len(df_llm)
print(df_llm.dtypes)

hippurate-mannitol            float64
Dibutylamine                  float64
hippuric acid                 float64
2-mercaptobenzothiazole       float64
N-acetyl-glucosaminic acid    float64
2-phenylacetamide             float64
lys-Ile/lys-leu               float64
y                              object
dtype: object


## Process LLM generated data to have the same data types

In [12]:
df_llm = df_llm.dropna()
df_llm = df_llm[~df_llm.apply(lambda row: any([isinstance(cell, str) and cell in ['integer', 'float', 'numeric', 'categorical', 'number', 'No', 'Yes', 'continuous', 'age in years', 'string'] for cell in row]), axis=1)]

example_df = deepcopy(X_train_orig)
example_df['y'] = deepcopy(y_train_orig)

try:
    df_llm = df_llm.astype(example_df.dtypes)
except:
    # Assuming the dtypes from the example_df['Dtrain'].dataframe() is what you want
    target_dtypes = example_df.dtypes.to_dict()

    problematic_rows = set()

    for col, dtype in target_dtypes.items():
        for index, value in df[col].items():
            try:
                _ = dtype.type(value)  # Try to convert the value
            except Exception:
                problematic_rows.add(index)

    # Convert the problematic rows to a list and sort them
    problematic_rows = sorted(list(problematic_rows))

    # Drop the problematic rows
    df_llm.drop(problematic_rows, inplace=True)

    # Identify rows where any cell is of type list
    rows_with_lists = df.applymap(lambda x: isinstance(x, list)).any(axis=1)

    # Drop those rows
    df_llm = df_llm[~rows_with_lists]

    df_llm = df_llm.astype(example_df.dtypes)


df_llm

,hippurate-mannitol,Dibutylamine,hippuric acid,2-mercaptobenzothiazole,N-acetyl-glucosaminic acid,2-phenylacetamide,lys-Ile/lys-leu,y
0,5.432110e+04,6789.012345,543.210987,234.123456,56789.01234,7.890123e+05,8.901235e+04,1
1,5.432110e+04,6789.123456,123.456789,56789.012340,123456.78910,2.345679e+06,3.456789e+06,1
2,1.456201e+06,316429.508400,9854.210330,3542.184210,278641.73120,2.345202e+06,1.806456e+06,1
3,7.836414e+05,55516.371460,7839.932526,2250.008480,79531.75333,6.029282e+05,7.173814e+05,1
4,1.195384e+06,91160.618530,15365.425440,900.389529,136703.07270,2.000293e+06,1.068308e+06,1
...,...,...,...,...,...,...,...,...
86,1.389283e+05,238430.477600,1095.420133,16590.667360,293454.51210,1.475067e+06,1.893961e+06,1
87,7.460122e+05,301611.136100,2511.791341,11327.215380,381246.07240,1.203403e+06,1.628105e+06,1
88,1.551939e+05,127500.783200,1747.820155,7591.357666,225639.19150,7.886709e+05,1.020119e+06,1
89,5.468361e+05,452034.358200,5801.856165,26329.945880,819308.59020,2.530293e+06,3.529588e+06,1


In [13]:
df_llm.dtypes

hippurate-mannitol            float64
Dibutylamine                  float64
hippuric acid                 float64
2-mercaptobenzothiazole       float64
N-acetyl-glucosaminic acid    float64
2-phenylacetamide             float64
lys-Ile/lys-leu               float64
y                               int64
dtype: object

# STEP 2: Curation

In [14]:
#from src.curation import data_centric_curation
from cllm.curation import data_centric_curation
X_check = df_llm.drop(columns=['y'])
y_check = df_llm['y'].values.astype(int)

curation_metric = 'aleatoric'
curation_ythresh=0.2
curation_xthresh=0 #adaptive

easy_train, ambig_train, unlearnable_train, Curator_xgb = data_centric_curation(X_train_orig, y_train_orig, X_check, y_check, 
                 curation_metric=curation_metric, retrain=False, nest = 100, 
                 curation_ythresh=curation_ythresh, curation_xthresh=curation_xthresh)

curated_train_ids = np.concatenate((easy_train, ambig_train))
curated_train_ids, unlearnable_train

Using adaptive threshold


(array([ 2,  6,  9, 10, 12, 14, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28,
        30, 32, 33, 35, 37, 38, 40, 41, 42, 43, 45, 46, 50, 51, 52, 53, 54,
        55, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
        73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 88, 89, 90,  0,
         1,  3,  5,  8, 11, 15, 16, 23, 29, 31, 34, 36, 39, 44, 47, 48, 49,
        56, 74, 85], dtype=int64),
 array([ 4,  7, 13], dtype=int64))

In [17]:
# Create a DataFrame with samples that pass curation (curated_train_ids)
df_curated = df_llm.iloc[curated_train_ids]

# Print information about the curated data
print(f"Shape of original generated data: {df_llm.shape}")
print(f"Shape of curated data: {df_curated.shape}")
print(f"Percentage of data retained after curation: {(len(curated_train_ids) / len(df_llm)) * 100:.2f}%")

# Display distribution of easy and ambiguous samples
print(f"\nNumber of easy samples: {len(easy_train)}")
print(f"Number of ambiguous samples: {len(ambig_train)}")
print(f"Number of unlearnable samples: {len(unlearnable_train)}")

# Calculate class distribution in curated data
y_curated = df_curated['y'].values
class_distribution = np.bincount(y_curated) / len(y_curated)
print("\nClass distribution in curated data:")
for i, prob in enumerate(class_distribution):
    print(f"Class {i}: {prob:.2f}")

# Optionally, save the curated data to a CSV file
df_curated.to_csv("rcc_samples.csv", index=False)
print("\nCurated samples saved to 'rcc_samples.csv'")


Shape of original generated data: (91, 8)
Shape of curated data: (88, 8)
Percentage of data retained after curation: 96.70%

Number of easy samples: 67
Number of ambiguous samples: 21
Number of unlearnable samples: 3

Class distribution in curated data:
Class 0: 0.00
Class 1: 1.00

Curated samples saved to 'rcc_samples.csv'


In [33]:
df_curated = df_curated.sample(n=needed_synthetic)

rcc = pd.read_csv("rcctrain.csv")
print(rcc.shape[0])
appendable_curated = df_curated.rename(columns={'y': 'Groups'})
print(appendable_curated.shape[0])
appendable_curated.to_csv("rcc_synthetic.csv")

balanced = pd.concat([rcc, appendable_curated], ignore_index=True)
print(balanced.shape[0])
balanced.to_csv("rcc_balanced.csv")

204
76
280


In [34]:
count_0 = (balanced['Groups'] == 0).sum()
count_1 = (balanced['Groups'] == 1).sum()

print(f"Count of y=0: {count_0}")
print(f"Count of y=1: {count_1}")
df_curated

Count of y=0: 140
Count of y=1: 140


,hippurate-mannitol,Dibutylamine,hippuric acid,2-mercaptobenzothiazole,N-acetyl-glucosaminic acid,2-phenylacetamide,lys-Ile/lys-leu,Groups
86,138928.30220,238430.477600,1095.420133,16590.667360,293454.51210,1.475067e+06,1.893961e+06,1
24,145633.30900,245643.078900,1098.543200,18906.789000,301098.54500,1.501099e+06,1.901891e+06,1
30,90023.12000,340212.315000,1001.123000,4563.234000,987000.12500,4.560000e+06,3.456000e+06,1
15,987654.32000,54321.110000,7654.320000,3456.780000,87654.32000,5.432111e+05,9.876543e+04,1
25,734091.32410,124384.441200,1423.548544,10232.985310,384291.24940,1.032139e+06,1.486128e+06,1
...,...,...,...,...,...,...,...,...
71,134819.21304,201554.231270,1129.648211,17623.534530,304521.48521,1.432145e+06,1.845322e+06,1
0,54321.09876,6789.012345,543.210987,234.123456,56789.01234,7.890123e+05,8.901235e+04,1
45,543910.12830,102354.821500,1283.762352,2891.320542,15687.46235,8.012351e+05,4.012867e+05,1
12,456789.12300,12345.123000,789.012000,6789.123000,13579.12300,8.901231e+05,1.234512e+04,1
